In [1]:
import numpy as np
import cv2

In [3]:
img = cv2.imread('./images/happy_parents.jpg')
cv2.imshow('happy parents',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
#load the model
face_detection_model = cv2.dnn.readNetFromCaffe('./model/deploy.prototxt.txt',
                                               './model/res10_300x300_ssd_iter_140000.caffemodel')

In [26]:
#Blob from image
image = img.copy()
face_mask = np.zeros(image.shape[:2], dtype=('uint8'))

blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,177,123),swapRB=True)
face_detection_model.setInput(blob)
detections = face_detection_model.forward()

h,w = image.shape[:2]
for i in range(detections.shape[2]):
    confidence = detections[0,0,i,2]
    if confidence > 0.5:
        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        box = box.astype(int)
        cv2.rectangle(face_mask,(box[0],box[1]),(box[2],box[3]),255,-1)
inv_face_mask = cv2.bitwise_not(face_mask)
#step-3 apply mask image to original image
bg = cv2.bitwise_and(image,image,mask=face_mask)
fg = cv2.bitwise_and(image,image,mask=inv_face_mask)
#step-4 blur bg_img with gaussian blur
bg_img_blur = cv2.GaussianBlur(bg, (51,51),0)
#step-5 addition
face_blur = cv2.add(fg,bg_img_blur)

cv2.imshow('original',image)
cv2.imshow('blur face', face_blur)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
detections.shape

(1, 1, 200, 7)